# testing

In [1]:
!pip install colabcode
!pip install fastapi
!pip install BaseModel

  Using cached BaseModel-20190604.1625.tar.gz (3.1 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
Discarding https://files.pythonhosted.org/packages/1f/83/134a871a36beb6e68b200c86b92ca6dc9583d55dc9f97e63bc5d1903eba2/BaseModel-20190604.1625.tar.gz (from https://pypi.org/simple/basemodel/): Requested BaseModel from https://files.pythonhosted.org/packages/1f/83/134a871a36beb6e68b200c86b92ca6dc9583d55dc9f97e63bc5d1903eba2/BaseModel-20190604.1625.tar.gz has inconsistent version: expected '20190604.1625', but metadata has '20221210.826'
  Using cached BaseModel-20190527.1004.tar.gz (3.0 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
Discarding https://files.pythonhosted.org/packages/1a/14/8e6f465fbd57e7457d136fbff64f1233b22d9ae308e22162014cfaf90088/BaseModel-20190527.1004.tar.gz (from https://pypi.org/simple/basemodel/): Requested BaseModel from https://files.pythonhosted.org

ERROR: Could not find a version that satisfies the requirement BaseModel (from versions: 20190515.1326, 20190515.1333, 20190520.928, 20190520.1135, 20190524.1622, 20190527.1004, 20190604.1625)
ERROR: No matching distribution found for BaseModel


In [2]:
!pip install uvicorn[standard]

In [3]:
!pip install tensorflow

In [4]:
!pip install keras

In [5]:
from colabcode import ColabCode
from fastapi import FastAPI
from pydantic import BaseModel
from fastapi.middleware.cors import CORSMiddleware

In [6]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
tf.__version__

'2.11.0'

In [7]:
class Image(BaseModel):
    imageName : str

In [ ]:
# Preprocessing the Training set
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('training_set',
                                                 target_size = (128, 128),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

In [9]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('test_set',
                                            target_size = (128, 128),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 235 images belonging to 2 classes.


In [ ]:
from sklearn.model_selection import train_test_split
train,test = train_test_split(training_set,test_set , train_size=0.7)

In [10]:
# Part 2 - Building the CNN

# Initialising the CNN
cnn = tf.keras.models.Sequential()

# Step 1 - Convolution
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[128, 128, 3]))

# Step 2 - Pooling
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# Adding a second convolutional layer
cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))


# Step 3 - Flattening
cnn.add(tf.keras.layers.Flatten())

# Step 4 - Full Connection
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

# Step 5 - Output Layer
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [11]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [12]:
# Training the CNN on the Training set and evaluating it on the Test set
cnn.fit(x = training_set, validation_data = test_set, epochs = 4)

Epoch 1/4
16/16 [==============================] - 22s 1s/step - loss: 0.9303 - accuracy: 0.6059 - val_loss: 0.7275 - val_accuracy: 0.6170
Epoch 2/4
16/16 [==============================] - 10s 647ms/step - loss: 0.6328 - accuracy: 0.6765 - val_loss: 0.6910 - val_accuracy: 0.6170
Epoch 3/4
16/16 [==============================] - 12s 729ms/step - loss: 0.6223 - accuracy: 0.6765 - val_loss: 0.6687 - val_accuracy: 0.6170
Epoch 4/4
16/16 [==============================] - 10s 628ms/step - loss: 0.6281 - accuracy: 0.6765 - val_loss: 0.6709 - val_accuracy: 0.6170


In [13]:
cnn.save("./myModel_v1")

INFO:tensorflow:Assets written to: ./myModel_v1\assets


INFO:tensorflow:Assets written to: ./myModel_v1\assets


In [14]:
cnn=tf.keras.models.load_model("./myModel_v1")

In [15]:
from keras.preprocessing import image
import numpy as np

In [16]:
#label
training_set.class_indices

{'CAN': 0, 'NOR': 1}

In [ ]:
from sklearn.metrics import confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
import itertools

X_test = np.concatenate((training_set, test_set), axis = 0)
Y_pred = test_set.predict(X_test)
def plot_confusion_matrix(cm, classes,  normalize=False,title='Confusion matrix',  cmap=plt.cm.Blues):
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=55)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

cm = confusion_matrix(np.argmax(test_set, axis=1), np.argmax(Y_pred, axis=1))

cm_plot_label =['benign', 'malignant']
plot_confusion_matrix(cm, cm_plot_label, title ='Confusion Metrix for Skin Cancer')